In [19]:
import unicodedata
import re
from tokenizers import Tokenizer, models, trainers, pre_tokenizers

In [20]:
file_path = "hi.txt file"

In [21]:
with open(file_path, "r", encoding="utf-8") as f:
    lines = f.readlines()

In [22]:
with open(file_path, "r", encoding="utf-8") as f:
    text = f.read()

unique_words = set(text.split())

print(f"Unique words in dataset: {len(unique_words)}")
print(len(text))


Unique words in dataset: 157875
18323067


In [23]:
print(lines[:5])

['हवा में उछालकर गोबर के उपले फेंकते हुए आई नजर, देखें वीडियो\n', ' जनता से रिश्ता वेबडेस्क। Desi Jugaad News: देसी जुगाड़ के मामले में भारतीयों जैसा हुनरमंद शायद ही कोई होता होगा. भारतीय लोग दुनियाभर में अपने टैलंट का जलवा दिखाते हैं. सोशल मीडिया (Social Media) पर आपको हमेशा कोई न कोई ऐसा वीडियो मिल जाएगा, जिसमें एक भारतीय अपने टैलेंट से लोगों का दिल जीत ले. जुगाड़ के मामले में भारत का किसी से भी कोई मुकाबला नहीं है. इन दिनों एक भारतीय महिला के हुनर का वीडियो जमकर वायरल हो रहा है. इस वीडियो में एक महिला गोबर से अपना टैलेंट दिखाती नजर आ रही है.हवा में उछालकर गोबर के उपले फेंकते हुए आई नजरवायरल हो रहे इस वीडियो में एक भारतीय महिला दीवार पर गोबर के उपले लगाती नजर आ रही है. जिस तरह से वह दीवार पर हवा में गोबर को फेंक-फेंककर उपले लगा रही है, उसे देखकर कोई अपनी आंखों पर भरोसा नहीं करेगा. महिला का निशाना देखकर आपको भी हैरानी होगी. आप देख सकते हैं कि महिला एकदम सटीक तरीके से उपलों को सही जगह पर लगा रही है. महिला का निशाना एकदम अचूक नजर आ रहा है. महिला का यह वीडियो सोशल मीडिया पर तेजी से वायरल

In [ ]:
def normalize_text(text):
    return unicodedata.normalize("NFKC", text)

def clean_text(text):
    text = re.sub(r'[^\u0900-\u097F\s?!।]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


def preprocess_corpus(lines):
    processed_lines = []
    for line in lines:
        line = normalize_text(line)
        line = clean_text(line)
        processed_lines.append(line)
    return processed_lines


processed_lines = preprocess_corpus(lines)

# Save the cleaned text
with open("hindi_corpus_cleaned.txt", "w", encoding="utf-8") as f:
    for line in processed_lines:
        f.write(line + "\n")

print(f"Preprocessing completed! {len(processed_lines)} clean sentences saved.")


Preprocessing completed! 20296 clean sentences saved.


In [ ]:
tokenizer = Tokenizer(models.BPE())

# trainer for BPE
trainer = trainers.BpeTrainer(vocab_size=150000, min_frequency=1, special_tokens=["<unk>", "<s>", "</s>"])

# Pre-tokenizer
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# Load Hindi dataset
files = ["hindi_corpus_cleaned.txt"]
tokenizer.train(files, trainer)

# Save tokenizer
tokenizer.save("hindi_tokenizer.json")

In [27]:
from tokenizers import Tokenizer

class HindiTokenizer:
    def __init__(self, model_path="hindi_tokenizer.json"):
        self.tokenizer = Tokenizer.from_file(model_path)

    def encode(self, text):
        return self.tokenizer.encode(text).ids

    def decode(self, token_ids):
        return self.tokenizer.decode(token_ids)


tokenizer = HindiTokenizer()

[1267, 2631, 14127, 4600, 4479, 658]


In [ ]:
tokenizer = Tokenizer.from_file("hindi_tokenizer.json")
vocab_size = tokenizer.get_vocab_size()
print(vocab_size)

110126


In [42]:
txt = "ऑनलाइन क्लास में हमेशा अपने पोस्चर (बैठने का आसन) को ठीक रखें। कभी झुककर नहीं बल्कि पीठ को हमेशा सीधा रखें।   Bokaro Coronavirus News Update"

In [43]:
tokenizer = HindiTokenizer()

In [44]:
ids = tokenizer.encode(txt)
print(ids)

[1793, 4081, 100, 1851, 285, 28931, 6799, 104, 9707, 102, 1318, 1918, 79, 1302, 55935, 179, 1681, 2855, 102, 1851, 4622, 1918, 79]


In [45]:
print(tokenizer.decode(ids))

ऑनलाइन क्लास में हमेशा अपने पोस्चर बैठने का आसन को ठीक रखें । कभी झुककर नहीं बल्कि पीठ को हमेशा सीधा रखें ।
